# Cargando Datos Consolidados del Dataset CoSiBD

Este notebook demuestra cómo cargar y utilizar los archivos consolidados del dataset de señales.

**Estructura del Dataset:**
- `signals_high_resolution_5000.[npz|txt|json]` - 2500 señales de 5000 muestras
- `signals_subsampled_simple_{size}.[npz|txt|json]` - Versiones submuestreadas (150, 250, 500, 1000)
- `signals_subsampled_filtered_{size}.[npz|txt|json]` - Versiones con filtro anti-aliasing
- `signals_metadata.json` - Metadata de todas las señales
- `dataset_summary.json` - Resumen del dataset
- `filtering_info.json` - Información sobre filtros aplicados

In [ ]:
# Importar librerías necesarias
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
from collections import Counter

# Configurar visualización
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1️⃣ Cargar Señales de Alta Resolución (5000 muestras)

Vamos a cargar el archivo NPZ que contiene todas las 2500 señales de alta resolución.

In [ ]:
# Ruta base del dataset
base_path = Path("SignalBuilderC/data")

# Cargar señales de alta resolución
data_hr = np.load(base_path / "signals_high_resolution_5000.npz")

# Extraer arrays
signals_hr = data_hr['signals']  # Señales con ruido
t_hr = data_hr['t']              # Array de tiempo
clean_signals_hr = data_hr['clean_signals']  # Señales sin ruido

print(f"📊 Dataset de Alta Resolución:")
print(f"  • Forma de signals: {signals_hr.shape}")
print(f"  • Forma de tiempo: {t_hr.shape}")
print(f"  • Número de señales: {signals_hr.shape[0]}")
print(f"  • Muestras por señal: {signals_hr.shape[1]}")
print(f"  • Señales limpias disponibles: {clean_signals_hr is not None}")
print(f"\n📈 Estadísticas:")
print(f"  • Media: {np.mean(signals_hr):.6f}")
print(f"  • Desviación estándar: {np.std(signals_hr):.6f}")
print(f"  • Min: {np.min(signals_hr):.6f}")
print(f"  • Max: {np.max(signals_hr):.6f}")

### Visualizar una señal individual

Accedemos a una señal específica por su índice (fila en el array).

In [ ]:
# Seleccionar señal por índice
signal_idx = 42
signal = signals_hr[signal_idx]
clean_signal = clean_signals_hr[signal_idx]

# Visualizar
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Señal con ruido
ax1.plot(t_hr, signal, 'b-', linewidth=0.8, alpha=0.8)
ax1.set_title(f'Señal {signal_idx} - Con Ruido', fontsize=14, fontweight='bold')
ax1.set_xlabel('Tiempo')
ax1.set_ylabel('Amplitud')
ax1.grid(True, alpha=0.3)

# Señal limpia
ax2.plot(t_hr, clean_signal, 'g-', linewidth=0.8)
ax2.set_title(f'Señal {signal_idx} - Sin Ruido (Clean)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Tiempo')
ax2.set_ylabel('Amplitud')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✅ Señal {signal_idx} visualizada (5000 muestras)")

## 2️⃣ Cargar Metadata de las Señales

El archivo `signals_metadata.json` contiene todos los parámetros de generación para cada señal.

In [ ]:
# Cargar metadata
with open(base_path / "signals_metadata.json", 'r') as f:
    metadata = json.load(f)

print(f"📋 Metadata cargada:")
print(f"  • Total de entradas: {len(metadata)}")
print(f"\n🔍 Metadata de la señal {signal_idx}:")

meta = metadata[signal_idx]
print(f"  • Signal ID: {meta['signal_id']}")
print(f"  • Seed: {meta['seed']}")
print(f"  • Índice: {meta['index']}")
print(f"  • Tipo de ruido: {meta['noise_profile']['noise_type']}")
print(f"  • Tipo de spline (amplitud): {meta['amplitude_spline_type']}")
print(f"  • Tau frecuencia: {meta['tau_frequency']:.4f}")
print(f"  • Tau amplitud: {meta['tau_amplitude']}")
print(f"  • Offset vertical: {meta['vertical_offset']:.4f}")

### Estadísticas del Dataset

Analicemos la distribución de tipos de ruido y splines en el dataset.

In [ ]:
# Contar distribución de tipos
noise_types = [m['noise_profile']['noise_type'] for m in metadata]
spline_types = [m['amplitude_spline_type'] for m in metadata]

noise_counts = Counter(noise_types)
spline_counts = Counter(spline_types)

# Visualizar
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Distribución de ruido
noise_labels = list(noise_counts.keys())
noise_values = list(noise_counts.values())
ax1.bar(noise_labels, noise_values, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax1.set_title('Distribución de Tipos de Ruido', fontsize=14, fontweight='bold')
ax1.set_ylabel('Número de Señales')
for i, v in enumerate(noise_values):
    ax1.text(i, v + 20, f'{v}\n({100*v/len(metadata):.1f}%)', 
             ha='center', va='bottom', fontweight='bold')

# Distribución de splines
spline_labels = list(spline_counts.keys())
spline_values = list(spline_counts.values())
ax2.bar(spline_labels, spline_values, color=['#d62728', '#9467bd'])
ax2.set_title('Distribución de Tipos de Spline (Amplitud)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Número de Señales')
for i, v in enumerate(spline_values):
    ax2.text(i, v + 20, f'{v}\n({100*v/len(metadata):.1f}%)', 
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 Distribuciones calculadas")

## 3️⃣ Cargar Señales para Super-Resolución

Cargamos pares de señales de baja y alta resolución para tareas de super-resolución.

In [ ]:
# Cargar señales de baja resolución (1000 muestras) y alta resolución (5000 muestras)
data_lr = np.load(base_path / "signals_subsampled_simple_1000.npz")
signals_lr = data_lr['signals']
t_lr = data_lr['t']

print(f"🎯 Pares para Super-Resolución:")
print(f"  • Baja resolución (LR): {signals_lr.shape}")
print(f"  • Alta resolución (HR): {signals_hr.shape}")
print(f"  • Factor de upsampling: {signals_hr.shape[1] // signals_lr.shape[1]}x")
print(f"\n✅ Cada señal LR corresponde a la misma señal HR en el mismo índice")

### Visualizar comparación LR vs HR

In [ ]:
# Comparar la misma señal en ambas resoluciones
idx = 100

plt.figure(figsize=(14, 6))
plt.plot(t_lr, signals_lr[idx], 'ro-', linewidth=1.5, markersize=3, 
         label=f'Baja Resolución (1000 muestras)', alpha=0.7)
plt.plot(t_hr, signals_hr[idx], 'b-', linewidth=0.8, 
         label=f'Alta Resolución (5000 muestras)', alpha=0.6)
plt.title(f'Comparación LR vs HR - Señal {idx}', fontsize=14, fontweight='bold')
plt.xlabel('Tiempo')
plt.ylabel('Amplitud')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"✅ Señal {idx} comparada: {signals_lr.shape[1]} → {signals_hr.shape[1]} muestras")

## 4️⃣ Crear División Train/Validation

Dividimos el dataset en conjuntos de entrenamiento y validación.

In [ ]:
# Función para crear split
def create_train_val_split(signals, train_ratio=0.8, random_seed=42):
    np.random.seed(random_seed)
    num_signals = signals.shape[0]
    indices = np.arange(num_signals)
    np.random.shuffle(indices)
    
    split_idx = int(num_signals * train_ratio)
    train_indices = indices[:split_idx]
    val_indices = indices[split_idx:]
    
    return signals[train_indices], signals[val_indices], train_indices, val_indices

# Crear splits (80% train, 20% validation)
train_hr, val_hr, train_idx, val_idx = create_train_val_split(signals_hr, 0.8, 42)
train_lr, val_lr, _, _ = create_train_val_split(signals_lr, 0.8, 42)

print(f"📚 División Train/Validation:")
print(f"\n  Entrenamiento:")
print(f"    • LR: {train_lr.shape}")
print(f"    • HR: {train_hr.shape}")
print(f"\n  Validación:")
print(f"    • LR: {val_lr.shape}")
print(f"    • HR: {val_hr.shape}")
print(f"\n  Proporción: {len(train_idx)}/{len(val_idx)} ({100*len(train_idx)/len(signals_hr):.0f}%/{100*len(val_idx)/len(signals_hr):.0f}%)")

## 5️⃣ Comparar Submuestreo Simple vs Filtrado

Comparamos las dos estrategias de submuestreo: re-evaluación simple vs filtro anti-aliasing.

In [ ]:
# Cargar ambas versiones (500 muestras)
data_simple = np.load(base_path / "signals_subsampled_simple_500.npz")
data_filtered = np.load(base_path / "signals_subsampled_filtered_500.npz")

signals_simple = data_simple['signals']
signals_filtered = data_filtered['signals']
t_500 = data_simple['t']

print(f"🔬 Comparación de métodos de submuestreo:")
print(f"  • Simple (re-evaluación): {signals_simple.shape}")
print(f"  • Filtrado (anti-aliasing): {signals_filtered.shape}")

In [ ]:
# Comparar señal específica
idx = 50

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Alta resolución (referencia)
axes[0].plot(t_hr, signals_hr[idx], 'k-', linewidth=0.8, alpha=0.6)
axes[0].set_title(f'Señal {idx} - Alta Resolución (5000 muestras)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Amplitud')
axes[0].grid(True, alpha=0.3)

# Submuestreo simple
axes[1].plot(t_500, signals_simple[idx], 'ro-', linewidth=1.2, markersize=2.5)
axes[1].set_title('Submuestreo Simple - Re-evaluación (500 muestras)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Amplitud')
axes[1].grid(True, alpha=0.3)

# Submuestreo filtrado
axes[2].plot(t_500, signals_filtered[idx], 'go-', linewidth=1.2, markersize=2.5)
axes[2].set_title('Submuestreo Filtrado - Anti-aliasing (500 muestras)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Tiempo')
axes[2].set_ylabel('Amplitud')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calcular diferencias
diff = np.abs(signals_simple[idx] - signals_filtered[idx])
print(f"\n📊 Diferencia entre métodos (señal {idx}):")
print(f"  • Diferencia media: {np.mean(diff):.6f}")
print(f"  • Diferencia máxima: {np.max(diff):.6f}")
print(f"  • Diferencia mínima: {np.min(diff):.6f}")

## 6️⃣ Cargar Múltiples Resoluciones

El dataset incluye 4 niveles de submuestreo. Carguemos todos para ver las diferencias.

In [ ]:
# Cargar todas las resoluciones
resolutions = [150, 250, 500, 1000, 5000]
all_signals = {}
all_times = {}

for res in resolutions:
    if res == 5000:
        all_signals[res] = signals_hr
        all_times[res] = t_hr
    else:
        data = np.load(base_path / f"signals_subsampled_simple_{res}.npz")
        all_signals[res] = data['signals']
        all_times[res] = data['t']

print(f"📦 Resoluciones cargadas:")
for res in resolutions:
    upsampling = 5000 // res
    print(f"  • {res} muestras: {all_signals[res].shape} (factor {upsampling}x)")

In [ ]:
# Visualizar todas las resoluciones de una misma señal
idx = 75

fig, axes = plt.subplots(5, 1, figsize=(14, 14))

for i, res in enumerate(resolutions):
    axes[i].plot(all_times[res], all_signals[res][idx], 
                 'o-' if res < 5000 else '-', 
                 linewidth=1.2 if res < 5000 else 0.8,
                 markersize=3 if res < 5000 else 0,
                 alpha=0.7)
    upsampling = 5000 // res
    axes[i].set_title(f'{res} muestras (factor {upsampling}×)', 
                      fontsize=11, fontweight='bold')
    axes[i].set_ylabel('Amplitud')
    axes[i].grid(True, alpha=0.3)
    
axes[-1].set_xlabel('Tiempo')
plt.suptitle(f'Señal {idx} en Diferentes Resoluciones', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print(f"✅ Señal {idx} visualizada en todas las resoluciones")

## 7️⃣ Resumen del Dataset

Información consolidada sobre el dataset completo.

In [ ]:
# Cargar resumen del dataset
with open(base_path / "dataset_summary.json", 'r') as f:
    summary = json.load(f)

print("=" * 80)
print("📊 RESUMEN DEL DATASET CoSiBD")
print("=" * 80)
print(f"\n🎯 Información General:")
print(f"  • Total de señales: {summary['num_signals']}")
print(f"  • Dominio temporal: [{summary['high_resolution']['t_start']}, {summary['high_resolution']['t_end']:.4f}]")
print(f"  • Frecuencia de muestreo alta: {summary['high_resolution']['fs']:.2f} Hz")
print(f"  • Muestras de alta resolución: {summary['high_resolution']['num_samples']}")

print(f"\n📦 Tamaños de submuestreo disponibles:")
for size in summary['subsample_sizes']:
    upsampling = 5000 // size
    print(f"  • {size} muestras (factor {upsampling}×)")

print(f"\n🔊 Configuración de ruido:")
print(f"  • Probabilidad: {summary['noise_config']['probability']*100:.0f}%")
print(f"  • Tipo: {summary['noise_config']['type']}")
print(f"  • STD relativa (Gaussiano): {summary['noise_config']['gaussian_std_relative']}")

print(f"\n🌱 Semillas:")
print(f"  • Semilla base: {summary['base_seed']}")
print(f"  • Rango: {summary['base_seed']} - {summary['base_seed'] + summary['num_signals'] - 1}")

print(f"\n📁 Estructura de archivos:")
print(f"  • Alta resolución: {summary['file_structure']['high_resolution']}")
print(f"  • Submuestreo simple: {summary['file_structure']['simple_subsampled']}")
print(f"  • Submuestreo filtrado: {summary['file_structure']['filtered_subsampled']}")
print(f"  • Metadata: {summary['file_structure']['metadata']}")

print(f"\n💾 Formatos de datos:")
for fmt, desc in summary['data_format'].items():
    print(f"  • {fmt}: {desc}")

print("=" * 80)

## ✅ Conclusión

Este notebook ha demostrado cómo:
1. ✅ Cargar señales de alta resolución desde archivos NPZ consolidados
2. ✅ Acceder a metadata de todas las señales
3. ✅ Cargar pares LR/HR para tareas de super-resolución
4. ✅ Crear divisiones train/validation
5. ✅ Comparar submuestreo simple vs filtrado
6. ✅ Trabajar con múltiples resoluciones simultáneamente
7. ✅ Entender la estructura completa del dataset

**Ventajas del formato consolidado:**
- 📦 Menos archivos (30 archivos vs ~67,500)
- ⚡ Carga más rápida (`np.load()` una sola vez)
- 🎯 Acceso directo por índice
- 💾 Mejor organización y transferencia